In [1]:
import json
import os

with open("./actors.jsonl", "r") as file:
    actors = [json.loads(line)[4]["response"]["actors"] for line in file]
actors[0][0]

{'active_skill': {'activate_skill_point': 2,
  'description': '造成敵方全體幹部40%ATK傷害，並在3回合內使其DEF-10%。\n',
  'image': '/images/active_skill/SKFRTK03_1.png',
  'level': 'I',
  'name': '阿剌模忒'},
 'actor_category_decor': '/images/actor_category/decor/cadrecardS.png',
 'actor_category_icon': '/images/actor_category/icon/cadre2.png',
 'actor_category_order': 30,
 'actor_prototype': {'active_skills': [{'activate_skill_point': 2,
    'background': '/images/actor_category/active_skill_background/S.png',
    'description': '造成敵方全體幹部40%ATK傷害，並在3回合內使其DEF-10%。\n',
    'id': 59,
    'image': '/images/active_skill/SKFRTK03_1.png',
    'level': 'I',
    'name': '阿剌模忒'}],
  'actor_category': {'font_color': '#65d56a', 'id': 2, 'name': '間諜'},
  'blood_base_value': 24000,
  'blood_elevation_value': 2250,
  'description': '八百年前阿拉莫忒堡陷落後，仍然在阿富汗山區祕密流傳的尼查里派（阿薩辛派，山中老人），不為世人所知。現在已經很少從事暗殺行動，但仍持續修習武術。為了躲避連年戰亂，從小就隱居在山洞裡修行，因此膚色偏白。學成以後，終於走向外面的世界…… ',
  'ethnic_group': '哈扎拉蒙古',
  'family': '阿富汗巴米揚',
  'id': 41,
  'name': '卡

In [2]:
# parse actors info from actors list
parsed_actors = {}
for actor in actors:
    for act in actor:
        if act["actor_prototype"]["name"] not in parsed_actors.keys():
            parsed_actors[act["actor_prototype"]["name"]] = {
                "name": act["actor_prototype"]["name"],
                "passive_skills" : act.get("passive_skills", []),
                "scarcity": act.get("scarcity", None),
                "nation": act.get("nation", None),
                "actor_category": act["actor_prototype"].get("actor_category", None).get("name", None),
                "weapon_name": act.get("no_weapon_name", None)
            }

In [3]:
parsed_actors

{'卡姆蘭': {'name': '卡姆蘭',
  'passive_skills': [{'description': ' 自身受到傷害-5%', 'level': 10},
   {'description': '間諜型成員攻擊力+5% 自身受到傷害-5%', 'level': 20},
   {'description': '間諜型成員攻擊力+5% 自身受到傷害-5%自身血量+ 5%', 'level': 30},
   {'description': '間諜型成員攻擊力+10% 自身受到傷害-5%自身血量+ 5%', 'level': 40},
   {'description': '間諜型成員攻擊力+10% 自身受到傷害5%自身血量+ 10%', 'level': 50},
   {'description': '間諜型成員攻擊力+15% 自身受到傷害-5%自身血量+ 10%', 'level': 60},
   {'description': '間諜型成員攻擊力+15% 自身受到傷害-5%自身血量+ 15%', 'level': 70},
   {'description': '間諜型成員攻擊力+20% 自身受到傷害-5%自身血量+ 15%', 'level': 80},
   {'description': '間諜型成員攻擊力+20% 自身受到傷害-5%自身血量+ 20%', 'level': 90},
   {'description': '間諜型成員攻擊力+25% 自身受到傷害-10%自身血量+ 20%', 'level': 100},
   {'description': '間諜型成員攻擊力+25% 自身受到傷害-10%自身血量+ 25%', 'level': 110},
   {'description': '間諜型成員攻擊力+30% 自身受到傷害-10%自身血量+ 25%', 'level': 120},
   {'description': '間諜型成員攻擊力+30% 自身受到傷害-10%自身血量+ 30%', 'level': 130},
   {'description': '間諜型成員攻擊力+35% 自身受到傷害-10%自身血量+ 30%', 'level': 140},
   {'description': '間諜型成員攻擊力+35

In [4]:
with open("./parsed_actors.json", "w", encoding="utf-8") as outfile:
    json.dump(parsed_actors, outfile, ensure_ascii=False, indent=4)

In [9]:
with open("./parsed_actors.json", "r") as outfile:
    parsed_actors = json.load(outfile)

In [10]:
passsive_skills_keys = {'自身': 'self', '型': 'category', '陣營' : 'nation'}
category_keys = list(set(parsed_actors[actor]['actor_category'] for actor in parsed_actors))
nation_keys = list(set(parsed_actors[actor]['nation'] for actor in parsed_actors))
stat_keys = {
    '攻擊力': 'atk',
    '受到傷害': 'def',
    '血量': 'hp',
}

In [11]:
nation_keys

['自由勢力', '嶺南海寇', '北元', '紅巾軍', '南明', '烏思藏', '東寧', '後金', '金帳汗國', '察合台汗國']

In [12]:
for actor_name, actor_data in parsed_actors.items():
    passive_skills = actor_data['passive_skills']
    parsed_passive_skills = {}
    for skill in passive_skills:
        skill_level = skill.get('level')
        skill_description = skill.get('description', '')
        tmp_passive_skills = {
            'self': { 'atk': 0, 'def': 0, 'hp': 0 },
            'category': { cat:{ 'atk': 0, 'def': 0, 'hp': 0 } for cat in category_keys },
            'nation': { nat:{ 'atk': 0, 'def': 0, 'hp': 0 } for nat in nation_keys },
        }
        # 使用 regex 對 description 分離出每個效果
        skill_description_list = skill_description.split('%')
        skill_description_list = [desc.strip() + '%' for desc in skill_description_list if desc.strip()]
        # print(skill_description_list)
        for passive_skill in skill_description_list:
            for key in passsive_skills_keys.keys():
                if key in passive_skill:
                    if key == '自身':
                        for stat_key in stat_keys.keys():
                            if stat_key in passive_skill:
                                if '+' in passive_skill:
                                    split_key = '+'
                                elif '-' in passive_skill:
                                    split_key = '-'
                                else:
                                    split_key = stat_key
                                print(passive_skill.split(split_key)[-1].strip()[:-1])
                                tmp_passive_skills[passsive_skills_keys[key]][stat_keys[stat_key]] = int(passive_skill.split(split_key)[-1].strip()[:-1])
                    elif key == '型':
                        for category in category_keys:
                            if category in passive_skill:
                                for stat_key in stat_keys.keys():
                                    if stat_key in passive_skill:
                                        if '+' in passive_skill:
                                            split_key = '+'
                                        elif '-' in passive_skill:
                                            split_key = '-'
                                        else:
                                            split_key = stat_key
                                        tmp_passive_skills[passsive_skills_keys[key]][category][stat_keys[stat_key]] = int(passive_skill.split(split_key)[-1].strip()[:-1])
                    elif key == '陣營':
                        for nation in nation_keys:
                            if nation in passive_skill:
                                for stat_key in stat_keys.keys():
                                    if stat_key in passive_skill:
                                        if '+' in passive_skill:
                                            split_key = '+'
                                        elif '-' in passive_skill:
                                            split_key = '-'
                                        else:
                                            split_key = stat_key
                                        tmp_passive_skills[passsive_skills_keys[key]][nation][stat_keys[stat_key]] = int(passive_skill.split(split_key)[-1].strip()[:-1])
        parsed_passive_skills[skill_level] = tmp_passive_skills
    parsed_actors[actor_name]['parsed_passive_skills'] = parsed_passive_skills
        

5
5
5
5
5
5
5
10
5
10
5
15
5
15
5
20
10
20
10
25
10
25
10
30
10
30
10
35
10
35
10
40
10
40
10
45
15
45
15
50
15
50
15
55
15
55
15
60
15
60
15
65
15
65
15
70
20
70
5
5
5
5
10
5
10
5
15
5
15
5
20
5
20
10
25
10
25
10
30
10
30
10
35
10
35
10
40
15
40
15
45
15
45
15
50
15
50
15
55
15
55
20
60
20
60
20
65
20
65
20
70
20
70
25
70
5
10
10
5
15
5
15
10
20
10
20
15
25
15
25
20
30
5
20
30
5
25
35
5
25
35
5
30
40
5
30
40
10
35
45
10
35
45
10
40
50
10
40
50
10
45
55
15
45
55
15
50
60
15
50
60
15
55
65
15
55
65
20
60
70
20
60
75
20
65
80
20
65
85
20
65
90
25
65
5
10
5
15
5
20
5
5
25
5
10
30
5
10
35
5
10
40
10
15
45
10
15
50
10
15
55
10
20
60
15
20
65
15
20
70
15
25
75
15
25
80
20
25
5
10
5
15
5
20
5
25
5
30
5
35
5
40
5
50
10
50
10
60
10
70
10
80
10
90
10
100
10
110
15
110
15
120
15
130
15
140
15
150
15
160
15
170
20
170
20
180
20
190
20
200
20
210
20
220
20
230
5
5
10
5
5
15
5
10
20
5
10
25
10
15
30
10
15
35
10
20
35
15
20
40
15
25
45
15
25
45
20
30
50
20
30
55
20
35
55
25
35
60
25
40
65
25
40
5
5
5

In [13]:
parsed_actors

{'卡姆蘭': {'name': '卡姆蘭',
  'passive_skills': [{'description': ' 自身受到傷害-5%', 'level': 10},
   {'description': '間諜型成員攻擊力+5% 自身受到傷害-5%', 'level': 20},
   {'description': '間諜型成員攻擊力+5% 自身受到傷害-5%自身血量+ 5%', 'level': 30},
   {'description': '間諜型成員攻擊力+10% 自身受到傷害-5%自身血量+ 5%', 'level': 40},
   {'description': '間諜型成員攻擊力+10% 自身受到傷害5%自身血量+ 10%', 'level': 50},
   {'description': '間諜型成員攻擊力+15% 自身受到傷害-5%自身血量+ 10%', 'level': 60},
   {'description': '間諜型成員攻擊力+15% 自身受到傷害-5%自身血量+ 15%', 'level': 70},
   {'description': '間諜型成員攻擊力+20% 自身受到傷害-5%自身血量+ 15%', 'level': 80},
   {'description': '間諜型成員攻擊力+20% 自身受到傷害-5%自身血量+ 20%', 'level': 90},
   {'description': '間諜型成員攻擊力+25% 自身受到傷害-10%自身血量+ 20%', 'level': 100},
   {'description': '間諜型成員攻擊力+25% 自身受到傷害-10%自身血量+ 25%', 'level': 110},
   {'description': '間諜型成員攻擊力+30% 自身受到傷害-10%自身血量+ 25%', 'level': 120},
   {'description': '間諜型成員攻擊力+30% 自身受到傷害-10%自身血量+ 30%', 'level': 130},
   {'description': '間諜型成員攻擊力+35% 自身受到傷害-10%自身血量+ 30%', 'level': 140},
   {'description': '間諜型成員攻擊力+35

In [14]:
with open("./parsed_actors_skill.json", "w", encoding="utf-8") as outfile:
    json.dump(parsed_actors, outfile, ensure_ascii=False, indent=4)